## MongoDB

### Test

In [1]:
import pymongo
import key_param

client = pymongo.MongoClient(key_param.MONGODB_URI)
db = client.sample_mflix

collection = db.movies

print(collection.find_one())

{'_id': ObjectId('573a1390f29313caabcd42e8'), 'plot': 'A group of bandits stage a brazen train hold-up, only to find a determined posse hot on their heels.', 'genres': ['Short', 'Western'], 'runtime': 11, 'cast': ['A.C. Abadie', "Gilbert M. 'Broncho Billy' Anderson", 'George Barnes', 'Justus D. Barnes'], 'poster': 'https://m.media-amazon.com/images/M/MV5BMTU3NjE5NzYtYTYyNS00MDVmLWIwYjgtMmYwYWIxZDYyNzU2XkEyXkFqcGdeQXVyNzQzNzQxNzI@._V1_SY1000_SX677_AL_.jpg', 'title': 'The Great Train Robbery', 'fullplot': "Among the earliest existing films in American cinema - notable as the first film that presented a narrative story to tell - it depicts a group of cowboy outlaws who hold up a train and rob the passengers. They are then pursued by a Sheriff's posse. Several scenes have color included - all hand tinted.", 'languages': ['English'], 'released': datetime.datetime(1903, 12, 1, 0, 0), 'directors': ['Edwin S. Porter'], 'rated': 'TV-G', 'awards': {'wins': 1, 'nominations': 0, 'text': '1 win.'},

## Load Data to Mongo db after had created embeddings

In [2]:
from pymongo import MongoClient
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.document_loaders import DirectoryLoader
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import gradio as gr
from gradio.themes.base import Base
import key_param

/Users/paolobonicco/Projects/PARASOL/fine-tuning-llm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
client = MongoClient(key_param.MONGODB_URI)
db_name = "parasol-2"
collection_name = "collection_of_text_blobs"
collection = client[db_name][collection_name]

In [4]:
loader = DirectoryLoader('./sample_files', glob='*.txt', show_progress=True)
data = loader.load()

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:02<00:00,  1.52it/s]


In [5]:
embeddigns = OpenAIEmbeddings(openai_api_key=key_param.OPENAI_API_KEY)

vectorStore = MongoDBAtlasVectorSearch.from_documents(data, embeddigns, collection=collection)

/Users/paolobonicco/Projects/PARASOL/fine-tuning-llm/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## RAG

In [6]:
client = MongoClient(key_param.MONGODB_URI)
db_name = "parasol-2"
collection_name = "collection_of_text_blobs"
collection = client[db_name][collection_name]

In [7]:
def query_data(query):
    docs = vectorStore.similarity_search(query, top_k=1) # extract docs from mongo with similar text
    as_output = docs[0].page_content # extract the text from the doc
    
    llm = OpenAI(openai_api_key=key_param.OPENAI_KEY, temperature=0)
    retriever = vectorStore.as_retriever() # Setup a retriever to search for similar documents
    
    # Langchain function to create a chain prompt that add some text to the query and create prompt
    qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)
    retriever_output = qa.run(query)
    
    llm_output = llm.invoke(query)
    
    return as_output, retriever_output, llm_output

### Gradio

In [8]:
with gr.Blocks(theme=Base(), title="Question Answering App using Vector Search + RAG") as demo:
  gr.Markdown(
    """"
    # Question Answerfing App using Atlas Vector Search + RAG Architecture
    """)
  textbox = gr. Textbox(label="Enter your Question:")
  with gr.Row():
    button = gr.Button ("Submit", variant="primary")
  with gr. Column ():
    output1 = gr.Textbox(lines=1, max_lines=10, label="Output with just Atlas Vector Search (return text field as is):")
    output2 = gr.Textbox(lines=1, max_lines=10, label="Output generated by chaining Atlas Vector Search to Langchain's RetrieverQa + OpenAi LLM:")
    output3 = gr.Textbox(lines=1, max_lines=10, label="Output generated LLM only:")

  button.click(query_data, textbox, outputs=[output1, output2, output3])
  
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/paolobonicco/Projects/PARASOL/fine-tuning-llm/.venv/lib/python3.11/site-packages/gradio/queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/paolobonicco/Projects/PARASOL/fine-tuning-llm/.venv/lib/python3.11/site-packages/gradio/route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/paolobonicco/Projects/PARASOL/fine-tuning-llm/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1689, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/paolobonicco/Projects/PARASOL/fine-tuning-llm/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1255, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/U